### Notes for cleaning:
- False investments to clean:
    - Per http://www.sharkalytics.com/company/classroom-jams - Mark Cuban was not present, caught 50000 investment under Mark name that is not true. Need to remove. - COMPLETE
    - Per http://www.sharkalytics.com/company/the-beebo - Barbara investment was actually Lori, Barabara wasn't present, pulled up show to confirm. - COMPLETE
    - Per http://www.sharkalytics.com/company/shark-wheel - Robert investment was actually Kevin. - COMPLETE
    - Per http://www.sharkalytics.com/company/pursecase - Daymon was not present, Lori actually made this deal. - COMPLETE

- Analysis of main sharks only, guest data/columns dropped.

In [41]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
from utils.util import check_nulls
from utils.data_load import make_id_column, merge_dataframes


In [2]:
df = pd.read_csv('datasets/Shark Tank US dataset.csv')

In [3]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [4]:
df.pitch_number.value_counts()

pitch_number
1       1
906     1
914     1
913     1
912     1
       ..
453     1
452     1
451     1
450     1
1361    1
Name: count, Length: 1361, dtype: int64

In [5]:
df.columns

Index(['season_number', 'startup_name', 'episode_number', 'pitch_number',
       'season_start', 'season_end', 'original_air_date', 'industry',
       'business_description', 'company_website', 'pitchers_gender',
       'pitchers_average_age', 'pitchers_city', 'pitchers_state',
       'entrepreneur_names', 'multiple_entrepreneurs', 'us_viewership',
       'original_ask_amount', 'original_offered_equity', 'valuation_requested',
       'got_deal', 'total_deal_amount', 'total_deal_equity', 'deal_valuation',
       'number_of_sharks_in_deal', 'investment_amount_per_shark',
       'equity_per_shark', 'royalty_deal', 'advisory_shares_equity', 'loan',
       'deal_has_conditions', 'barbara_corcoran_investment_amount',
       'barbara_corcoran_investment_equity', 'mark_cuban_investment_amount',
       'mark_cuban_investment_equity', 'lori_greiner_investment_amount',
       'lori_greiner_investment_equity', 'robert_herjavec_investment_amount',
       'robert_herjavec_investment_equity', 'daymon

In [6]:
check_nulls(df)

{'season_number': 0,
 'startup_name': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'episode_number': 0,
 'pitch_number': 0,
 'season_start': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'season_end': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'original_air_date': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'industry': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'business_description': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'company_website': {'standard_nulls': 758,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'pitchers_gender': {'standard_nulls': 7,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'pitchers_average_age': {'standard_nulls'

In [7]:
sharks = [x for x in df.columns if 'present' in x]
investment_columns = [x for x in df.columns if 'investment_amount' in x]

investment_columns.remove('investment_amount_per_shark')
investment_columns.remove('guest_investment_amount')
sharks.remove('guest_present')

shark_investments = list(zip(sharks,investment_columns))
shark_investments

[('barbara_corcoran_present', 'barbara_corcoran_investment_amount'),
 ('mark_cuban_present', 'mark_cuban_investment_amount'),
 ('lori_greiner_present', 'lori_greiner_investment_amount'),
 ('robert_herjavec_present', 'robert_herjavec_investment_amount'),
 ('daymond_john_present', 'daymond_john_investment_amount'),
 ('kevin_o_leary_present', 'kevin_o_leary_investment_amount')]

In [8]:
# Before Fix
for x in shark_investments:
    print(df.loc[df['got_deal'] == True].groupby(x[0])[x[1]].mean().values[0], x[0])
    print(df.loc[df['got_deal'] == True].groupby(x[0])[x[1]].sum().values[0], x[1])

100000.0 barbara_corcoran_present
100000.0 barbara_corcoran_investment_amount
50000.0 mark_cuban_present
50000.0 mark_cuban_investment_amount
nan lori_greiner_present
0.0 lori_greiner_investment_amount
75000.0 robert_herjavec_present
75000.0 robert_herjavec_investment_amount
55000.0 daymond_john_present
55000.0 daymond_john_investment_amount
nan kevin_o_leary_present
0.0 kevin_o_leary_investment_amount


In [9]:
# REMOVED ROW 6 MARK CUBAN FALSE INVESTMENT of 50000
df.loc[(df['got_deal'] == True)&(df['mark_cuban_investment_amount'] == 50000)&(df['mark_cuban_present'] == False)][investment_columns]
df.loc[6, 'mark_cuban_investment_amount'] = 0

In [10]:
# MOVED ROW 495 BARBARA INVESTMENT TO LORI
df.loc[(df['got_deal'] == True)&(df['barbara_corcoran_investment_amount'] == 100000)&(df['barbara_corcoran_present'] == False)][investment_columns]
df.loc[495, 'barbara_corcoran_investment_amount'] = 0
df.loc[495, 'lori_greiner_investment_amount'] = 100000
df.loc[495, 'barbara_corcoran_investment_equity'] = 0
df.loc[495, 'lori_greiner_investment_equity'] = 15.0


In [11]:
# MOVED ROW 491 ROBERT INVESTMENT TO KEVIN
df.loc[(df['got_deal'] == True)&(df['robert_herjavec_investment_amount'] == 75000)&(df['robert_herjavec_present'] == False)][investment_columns]
df.loc[491, 'robert_herjavec_investment_amount'] = 0
df.loc[491, 'kevin_o_leary_investment_amount'] = 75000
df.loc[491, 'robert_herjavec_investment_equity'] = 0
df.loc[491, 'kevin_o_leary_investment_equity'] = 2.5



In [12]:
# MOVED ROW 301 DAYMOND INVESTMENT TO LORI
df.loc[(df['got_deal'] == True)&(df['daymond_john_investment_amount'] == 55000)&(df['daymond_john_present'] == False)][investment_columns]
df.loc[301, 'daymond_john_investment_amount'] = 0
df.loc[301, 'lori_greiner_investment_amount'] = 55000
df.loc[301, 'daymond_john_investment_equity'] = 0
df.loc[301, 'lori_greiner_investment_equity'] = 15.0




In [13]:
# AFTER FIX
for x in shark_investments:
    print(df.loc[df['got_deal'] == True].groupby(x[0])[x[1]].mean().values[0], x[0])
    print(df.loc[df['got_deal'] == True].groupby(x[0])[x[1]].sum().values[0], x[1])
    

0.0 barbara_corcoran_present
0.0 barbara_corcoran_investment_amount
0.0 mark_cuban_present
0.0 mark_cuban_investment_amount
nan lori_greiner_present
0.0 lori_greiner_investment_amount
0.0 robert_herjavec_present
0.0 robert_herjavec_investment_amount
0.0 daymond_john_present
0.0 daymond_john_investment_amount
nan kevin_o_leary_present
0.0 kevin_o_leary_investment_amount


In [14]:
check_nulls(df)

{'season_number': 0,
 'startup_name': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'episode_number': 0,
 'pitch_number': 0,
 'season_start': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'season_end': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'original_air_date': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'industry': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'business_description': {'standard_nulls': 0,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'company_website': {'standard_nulls': 758,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'pitchers_gender': {'standard_nulls': 7,
  'empty_strings': 0,
  'specific_values': 0,
  'whitespace_only': 0},
 'pitchers_average_age': {'standard_nulls'

In [15]:
guest_columns = [x for x in df.columns if 'guest' in x]
guest_columns

['guest_investment_amount',
 'guest_investment_equity',
 'guest_name',
 'guest_present']

In [16]:
df.drop(guest_columns, axis=1, inplace=True)

In [17]:
guest_columns = [x for x in df.columns if 'guest' in x]
guest_columns

[]

### DEALS TABLE --- PENDING
##### PENDING ITEMS
- NEED PROD TABLE FOR MERGE for prod ID
    - NEED TO DROP PROD_TITLE COL AFTER MERGE
- NEED EPI TABLE TO MERGE for epi ID
    - NEED TO DROP EPI NUM COL AFTER MERGE

##### COLS: 
- deal_id INTEGER PK ----- CREATED
- prod_id INTEGER FK >- products.prod_id ---- NEED PROD TABLE FOR MERGE 
- episode_id INTEGER FK >- episodes.episode_id ---------- NEED EPI TABLE TO MERGE 
- made_deal BOOLEAN ---- CONVERTED
- ask_req INTEGER
- equity_req SMALLINT
- valuation_req INTEGER
- deal_amount INTEGER
- deal_equity SMALLINT
- deal_valuation INTEGER
- per_shark_investment INTEGER
- per_shark_equity SMALLINT
- deal_air_date DATE ----- CONVERTED



In [83]:
# Splicing columns for this table

deals_df = df[['got_deal', 'original_ask_amount', 'original_offered_equity', 'valuation_requested', 'total_deal_amount', 'total_deal_equity', 'deal_valuation', 'investment_amount_per_shark', 'equity_per_shark', 'original_air_date', 'startup_name', 'episode_number']].copy()

In [84]:
deals_df.dtypes

got_deal                         int64
original_ask_amount              int64
original_offered_equity        float64
valuation_requested              int64
total_deal_amount              float64
total_deal_equity              float64
deal_valuation                 float64
investment_amount_per_shark    float64
equity_per_shark               float64
original_air_date               object
startup_name                    object
episode_number                   int64
dtype: object

In [85]:
# checking for any potential missing values where they shouldnt be
deals_df.loc[deals_df['got_deal'] == 1].isnull().any()

got_deal                       False
original_ask_amount            False
original_offered_equity        False
valuation_requested            False
total_deal_amount              False
total_deal_equity              False
deal_valuation                 False
investment_amount_per_shark    False
equity_per_shark               False
original_air_date              False
startup_name                   False
episode_number                 False
dtype: bool

In [86]:
# checking for any potential missing values where they shouldnt be

deals_df.loc[deals_df['got_deal'] == 0].isnull().any()

got_deal                       False
original_ask_amount            False
original_offered_equity        False
valuation_requested            False
total_deal_amount               True
total_deal_equity               True
deal_valuation                  True
investment_amount_per_shark     True
equity_per_shark                True
original_air_date              False
startup_name                   False
episode_number                 False
dtype: bool

In [87]:
deals_df['got_deal'] = deals_df['got_deal'].astype(bool)
deals_df['original_air_date'] = pd.to_datetime(deals_df['original_air_date'], format="%d-%b-%y")

In [88]:
deals_df.dtypes

got_deal                                 bool
original_ask_amount                     int64
original_offered_equity               float64
valuation_requested                     int64
total_deal_amount                     float64
total_deal_equity                     float64
deal_valuation                        float64
investment_amount_per_shark           float64
equity_per_shark                      float64
original_air_date              datetime64[ns]
startup_name                           object
episode_number                          int64
dtype: object

In [89]:
deals_df.original_air_date

0      2009-08-09
1      2009-08-09
2      2009-08-09
3      2009-08-09
4      2009-08-09
          ...    
1356   2024-04-19
1357   2024-05-03
1358   2024-05-03
1359   2024-05-03
1360   2024-05-03
Name: original_air_date, Length: 1361, dtype: datetime64[ns]

In [90]:
deals_df = make_id_column(deals_df, 'deal_id')

In [92]:
deals_df

,deal_id,got_deal,original_ask_amount,original_offered_equity,valuation_requested,total_deal_amount,total_deal_equity,deal_valuation,investment_amount_per_shark,equity_per_shark,original_air_date,startup_name,episode_number
0,1,True,50000,15.0,333333,50000.0,55.0,90909.0,50000.0,55.0,2009-08-09,AvaTheElephant,1
1,2,True,460000,10.0,4600000,460000.0,50.0,920000.0,230000.0,25.0,2009-08-09,Mr.Tod'sPieFactory,1
2,3,False,1200000,10.0,12000000,NaN,NaN,NaN,NaN,NaN,2009-08-09,Wispots,1
3,4,False,250000,25.0,1000000,NaN,NaN,NaN,NaN,NaN,2009-08-09,CollegeFoxesPackingBoxes,1
4,5,False,1000000,15.0,6666667,NaN,NaN,NaN,NaN,NaN,2009-08-09,IonicEar,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,1357,True,1500000,5.0,30000000,1500000.0,9.0,16666667.0,750000.0,4.5,2024-04-19,RoboBurger,21
1357,1358,False,100000,10.0,1000000,NaN,NaN,NaN,NaN,NaN,2024-05-03,SipHerbals,22
1358,1359,True,500000,2.0,25000000,500000.0,3.5,14285714.0,500000.0,3.5,2024-05-03,mosh,22
1359,1360,False,500000,4.0,12500000,NaN,NaN,NaN,NaN,NaN,2024-05-03,ARBER,22


### SHARKS TABLE - COMPLETE

##### COLS:
- shark_id INTEGER PK
- shark_first_name VARCHAR(25) --- CREATED
- shark_last_name VARCHAR(25)  --- CREATED


In [93]:
sharks

['barbara_corcoran_present',
 'mark_cuban_present',
 'lori_greiner_present',
 'robert_herjavec_present',
 'daymond_john_present',
 'kevin_o_leary_present']

In [95]:
shark_names = []
for x in sharks:
    split_names = x.split('_')
    split_names.remove('present')
    shark_names.append(split_names)
    
for x in shark_names:
    if len(x) > 2:
        x.remove('o')
    
shark_names

[['barbara', 'corcoran'],
 ['mark', 'cuban'],
 ['lori', 'greiner'],
 ['robert', 'herjavec'],
 ['daymond', 'john'],
 ['kevin', 'leary']]

In [96]:
sharks_df = pd.DataFrame(shark_names, columns=['shark_first_name', 'shark_last_name'])

In [97]:
sharks_df = make_id_column(sharks_df, 'shark_id')
sharks_df

,shark_id,shark_first_name,shark_last_name
0,1,barbara,corcoran
1,2,mark,cuban
2,3,lori,greiner
3,4,robert,herjavec
4,5,daymond,john
5,6,kevin,leary
